In [3]:
from exchelp.exception_helper import ReportObject, CoreException, object_from_module

import os
#
# Model Parameters
#

main_report = ReportObject().adddata('_title_',os.path.basename('rfc.ipynb')).\
    adddata('initialize_info',ReportObject().\
        adddata('source',r"data/raw/bbc4").\
        adddata('extension','.txt').\
        adddata('type','text-document')
    ).\
    adddata('cleanse_info',ReportObject().\
        adddata('char_cleaner_function',ReportObject().\
            adddata('module','aistudio.data.text_utils').\
            adddata('object','clear_default_chars')

        ).\
        adddata('stemming_lemmatization_function',ReportObject().\
            adddata('module','aistudio.data.text_utils').\
            adddata('object','stemming_lematization')
        ).\
        adddata('stemmer_or_lemmatizer_instance',ReportObject().\
                adddata('module','nltk.stem').\
                adddata('object','WordNetLemmatizer')
            )
    ).\
    adddata('prepare_info',ReportObject().\
        adddata('vectorizer_instance',ReportObject().\
            adddata('module','sklearn.feature_extraction.text').\
            adddata('object','CountVectorizer').\
            adddata('hyper_params',ReportObject().\
                adddata('max_features',1500).\
                adddata('min_df',5).\
                adddata('max_df',0.7)
            ).\
            adddata('stop_words_function',ReportObject().\
                    adddata('module','nltk.corpus').\
                    adddata('object','stopwords').\
                    adddata('subObject','words')
            ).\
            adddata('language','english')
        ).\
        adddata('transformer_instance',ReportObject().\
            adddata('module','sklearn.feature_extraction.text').\
            adddata('object','TfidfTransformer')
        ).\
        adddata('test_size',0.2).\
        adddata('random_state',0)
    ).\
    adddata('execute_info',ReportObject().\
        adddata('model_instance',ReportObject().\
            adddata('module','sklearn.ensemble').
            adddata('object','RandomForestClassifier').\
            adddata('hyper_params',ReportObject().\
                adddata('n_estimators',1000).\
                adddata('random_state',0)
            )
        )
    )

In [4]:
from sklearn.datasets import  load_files
import numpy as np

#
# Initialize
#
data_source = main_report.getdata('initialize_info')['source']
print(data_source)
try:
    movie_data = load_files(data_source)
except Exception as e:
    CoreException('data load failed',e,logIt=True,dontThrow=True,shouldExit=True).act()
X, y = np.array(movie_data.data), np.array(movie_data.target)

main_report.getdata('initialize_info')['raw_number']=len(X)


{
 "message": "data load failed",
 "dontthrow": true,
 "logit": true,
 "_file": "/var/folders/c9/912h9yrs5jbdsvklgxszk8br0000gn/T/ipykernel_16192/1631172714.py",
 "_line": 9,
 "shouldexit": true,
 "_timeStamp": "2022-12-23 17:20:43.712652",
 "_env": "oe-ws-main.local",
 "_class": "CoreException",
 "_cause": {
  "_repr": "FileNotFoundError(2, 'No such file or directory')"
 },
 "_repr": "CoreException('data load failed', FileNotFoundError(2, 'No such file or directory'))"
}


NameError: name 'exit' is not defined

In [3]:
#
# Cleanse
#
from aistudio.reflection import runtime_objects

#char cleaning
moduleName = main_report.getData('cleanse_info')['char_cleaner_function']['module']
objectName = main_report.getData('cleanse_info')['char_cleaner_function']['object']
docs = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName)(data=X)

#stemming-lemmatization cleaning
moduleName = main_report.getData('cleanse_info')['stemmer_or_lemmatizer_instance']['module']
objectName = main_report.getData('cleanse_info')['stemmer_or_lemmatizer_instance']['object']
stem_or_lemmatizer_instance = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName)()

moduleName = main_report.getData('cleanse_info')['stemming_lemmatization_function']['module']
objectName = main_report.getData('cleanse_info')['stemming_lemmatization_function']['object']
docs = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName)(stem_or_lemmatizer=stem_or_lemmatizer_instance,data=X)

In [4]:
#
# Prepare
#

#   stopwords
moduleName = main_report.getData('prepare_info')['vectorizer_instance']['stop_words_function']['module']
objectName = main_report.getData('prepare_info')['vectorizer_instance']['stop_words_function']['object']
subObjectName = main_report.getData('prepare_info')['vectorizer_instance']['stop_words_function']['subObject']
stop_words_language = main_report.getData('prepare_info')['vectorizer_instance']['language']
stop_words = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName,subObjectName=subObjectName)(stop_words_language)

In [5]:
#   vectorizer
vectorizer_instance_params = main_report.getData('prepare_info')['vectorizer_instance']['hyper_params']
moduleName = main_report.getData('prepare_info')['vectorizer_instance']['module']
objectName = main_report.getData('prepare_info')['vectorizer_instance']['object']
vectorizer_instance = runtime_objects.object_from_module(moduleName,objectName)(**vectorizer_instance_params,stop_words = stop_words)

vectorized_count = vectorizer_instance.fit_transform(docs).toarray()

In [6]:
#   transformer
moduleName = main_report.getData('prepare_info')['transformer_instance']['module']
objectName = main_report.getData('prepare_info')['transformer_instance']['object']
transformer_instance = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName)()

transformed_vector = transformer_instance.fit_transform(vectorized_count).toarray()

test_size = objectName = main_report.getData('prepare_info')['test_size']
random_state = objectName = main_report.getData('prepare_info')['random_state']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_vector, y, test_size=test_size, random_state=random_state)


In [7]:
#
# Execute
#
moduleName = main_report.getData('execute_info')['model_instance']['module']
objectName = main_report.getData('execute_info')['model_instance']['object']
model_instance_parameters = main_report.getData('execute_info')['model_instance']['hyper_params']
model_instance = runtime_objects.object_from_module(moduleName=moduleName,objectName=objectName)(**model_instance_parameters)
model_instance.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [10]:
#
# Reportize
#
from sklearn.metrics import classification_report
import sys


y_pred = model_instance.predict(X_test)
target_names = ['business', 'entertainment', 'politics','sport','tech']
classification_performance = classification_report(y_test, y_pred, target_names=target_names,output_dict=True)
print(classification_report(y_test, y_pred, target_names=target_names))


model_name = main_report.getData('execute_info')['model_instance']['object']+'.sav'
run_title = main_report.getData('_title_')

main_report.addData('metric_info',rep_util.ReportObject().\
    addData('model_name',model_name).\
    addData('model_size',sys.getsizeof(model_instance)).\
    addData('target_names',target_names).\
    addData('summary',classification_performance)
    )

               precision    recall  f1-score   support

     business       0.94      0.93      0.94       102
entertainment       0.98      0.94      0.96        84
     politics       0.95      0.92      0.94        78
        sport       0.97      1.00      0.99       103
         tech       0.95      0.99      0.97        78

     accuracy                           0.96       445
    macro avg       0.96      0.96      0.96       445
 weighted avg       0.96      0.96      0.96       445



ReportObject()

In [11]:
#
#
# Persist

rep_util.persist_model(model_name,model_instance,main_report,run_title)